# 基于Transformers的多项选择

## Step1 导入相关包

In [3]:
import evaluate
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer

## Step2 加载数据集

In [4]:
c3 = DatasetDict.load_from_disk("./c3/")
c3

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 1625
    })
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

In [5]:
c3["train"][:10]

{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'context': [['男：你今天晚上有时间吗?我们一起去看电影吧?', '女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……'],
  ['男：足球比赛是明天上午八点开始吧?', '女：因为天气不好，比赛改到后天下午三点了。'],
  ['女：今天下午的讨论会开得怎么样?', '男：我觉得发言的人太少了。'],
  ['男：我记得你以前很爱吃巧克力，最近怎么不吃了，是在减肥吗?', '女：是啊，我希望自己能瘦一点儿。'],
  ['女：过几天刘明就要从英国回来了。我还真有点儿想他了，记得那年他是刚过完中秋节走的。',
   '男：可不是嘛!自从我去日本留学，就再也没见过他，算一算都五年了。',
   '女：从2000年我们在学校第一次见面到现在已经快十年了。我还真想看看刘明变成什么样了!',
   '男：你还别说，刘明肯定跟英国绅士一样，也许还能带回来一个英国女朋友呢。'],
  ['男：好久不见了，最近忙什么呢?',
   '女：最近我们单位要搞一个现代艺术展览，正忙着准备呢。',
   '男：你们不是出版公司吗?为什么搞艺术展览?',
   '女：对啊，这次展览是我们出版的一套艺术丛书的重要宣传活动。'],
  ['男：会议结束后，你记得把空调和灯都关了。', '女：好的，我知道了，明天见。'],
  ['男：你出国读书的事定了吗?', '女：思前想后，还拿不定主意呢。'],
  ['男：这件衣服我要了，在哪儿交钱?', '女：前边右拐就有一个收银台，可以交现金，也可以刷卡。'],
  ['男：小李啊，你是我见过的最爱干净的学生。',
   '女：谢谢教授夸奖。不过，您是怎么看出来的?',
   '男：不管我叫你做什么，你总是推得干干净净。',
   '女：教授，我……']],
 'question': ['女的最喜欢哪种电影?',
  '根据对话，可以知道什么?',
  '关于这次讨论会，我们可以知道什么?',
  '女的为什么不吃巧克力了?',
  '现在大概是哪一年?',
  '女的的公司为什么要做现代艺术展览?',
  '他们最可能是什么关系?',
  '女的是什么意思?',
  '他们最可能在什么地方?',
  '教授认为小李怎么样?'],
 'choi

In [6]:
c3.pop("test")

Dataset({
    features: ['id', 'context', 'question', 'choice', 'answer'],
    num_rows: 1625
})

In [7]:
c3

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

## Step3 数据集预处理

In [8]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [9]:
def process_function(examples): 
    # examples, dict, keys: ["context", "quesiton", "choice", "answer"]
    # examples, 1000
    context = []
    question_choice = []
    labels = []
    for idx in range(len(examples["context"])):
        ctx = "\n".join(examples["context"][idx])
        question = examples["question"][idx]
        choices = examples["choice"][idx]
        for choice in choices:
            context.append(ctx)
            question_choice.append(question + " " + choice)
        if len(choices) < 4:
            for _ in range(4 - len(choices)):
                context.append(ctx)
                question_choice.append(question + " " + "不知道")
        labels.append(choices.index(examples["answer"][idx]))
    tokenized_examples = tokenizer(context, question_choice, truncation="only_first", max_length=256, padding="max_length")     # input_ids: 4000 * 256, 
    tokenized_examples = {k: [v[i: i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}     # 1000 * 4 *256
    tokenized_examples["labels"] = labels
    return tokenized_examples
        

In [ ]:
res = c3["train"].select(range(10)).map(process_function, batched=True)
res

In [ ]:
import numpy as np
np.array(res["input_ids"]).shape

In [ ]:
tokenized_c3 = c3.map(process_function, batched=True)
tokenized_c3

## Step4 创建模型

In [ ]:
model = AutoModelForMultipleChoice.from_pretrained("hfl/chinese-macbert-base")

## Step5 创建评估函数

In [ ]:
import numpy as np
accuracy = evaluate.load("accuracy")

def compute_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

## Step6 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./muliple_choice",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True
)

## Step7 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_c3["train"],
    eval_dataset=tokenized_c3["validation"],
    compute_metrics=compute_metric
)

## Step8 模型训练

In [ ]:
trainer.train()

## Step9 模型预测

In [ ]:
from typing import Any
import torch


class MultipleChoicePipeline:

    def __init__(self, model, tokenizer) -> None:
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device

    def preprocess(self, context, quesiton, choices):
        cs, qcs = [], []
        for choice in choices:
            cs.append(context)
            qcs.append(quesiton + " " + choice)
        return tokenizer(cs, qcs, truncation="only_first", max_length=256, return_tensors="pt")

    def predict(self, inputs):
        inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()}
        return self.model(**inputs).logits

    def postprocess(self, logits, choices):
        predition = torch.argmax(logits, dim=-1).cpu().item()
        return choices[predition]

    def __call__(self, context, question, choices) -> Any:
        inputs = self.preprocess(context, question, choices)
        logits = self.predict(inputs)
        result = self.postprocess(logits, choices)
        return result

In [ ]:
pipe = MultipleChoicePipeline(model, tokenizer)

In [ ]:
pipe("小明在北京上班", "小明在哪里上班？", ["北京", "上海", "河北", "海南", "河北", "海南"])